# Save basis EZG data from PIA and hydrosheds as geojsons to Station output folders

In [2]:
from camelsp import Station, get_metadata
import geopandas as gpd
from glob import glob

## Basis EZG (Pia)

In [7]:
# get metadata
meta = get_metadata()

# create a list of camels ids
camels_ids = meta["camels_id"].tolist()

# get the geopackage with all station catchments
gdf_all = gpd.read_file('../catchments_pia/camels_20231018/camels_catchments_n2860.gpkg')

for camels_id in camels_ids:
    try:
        # initiate Station
        s = Station(camels_id)

        # read geojson file
        gdf = gdf_all[gdf_all['camels_id'] == camels_id].reset_index()

        # save geojson to Station output folder
        s.save_catchment_geometry(gdf, datasource='basis_ezg')

    except Exception as e:
        print(f"{camels_id} --- Error: {e}")


DEF10060 --- Error: catchment_geometry contains more than one geometries / rows for the station
DEF10780 --- Error: catchment_geometry contains more than one geometries / rows for the station
DEF14180 --- Error: catchment_geometry contains more than one geometries / rows for the station
DED10050 --- Error: catchment_geometry contains more than one geometries / rows for the station
DED10110 --- Error: catchment_geometry contains more than one geometries / rows for the station
DED10190 --- Error: catchment_geometry contains more than one geometries / rows for the station
DED10240 --- Error: catchment_geometry contains more than one geometries / rows for the station
DED10260 --- Error: catchment_geometry contains more than one geometries / rows for the station
DED10370 --- Error: catchment_geometry contains more than one geometries / rows for the station
DED10380 --- Error: catchment_geometry contains more than one geometries / rows for the station
DED10480 --- Error: catchment_geometry c

## HydroSheds

In [8]:
hydrosheds = glob('../hydrosheds/*.geojson')

for hydroshed in hydrosheds:
    try:
        # get the id
        id = hydroshed.split('/')[-1].split('_')[0]
        # initiate Station
        s = Station(hydroshed.split('/')[-1].split('_')[0])

        # read geojson file
        gdf = gpd.read_file(hydroshed)

        # save geojson to Station output folder
        s.save_catchment_geometry(gdf, datasource='hydrosheds')

    except Exception as e:
        print(f"{hydroshed} --- Error: {e}")

../hydrosheds/DE812120_hydrosheds.geojson --- Error: index 0 is out of bounds for axis 0 with size 0


In [5]:
hydrosheds[0].split('/')[-1].split('_')[0]

'DE412290'